In [2]:
!pip3 install selenium

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from json import dumps
import pandas as pd

In [4]:
#lendo arquivo csv com os tickers e salvando em um arquivo csv com os status

#lendo csv original
tabela = pd.read_csv("statusinvest-busca-avancada.csv", sep=";")

#criando dataframe com apenas com os códigos
df_ticker =  pd.DataFrame(tabela["TICKER"])

#inserindo coluna com status
df_ticker.insert(loc=0, column='Status', value=0)

#salvando o arquivo csv apenas com código e status
df_ticker.to_csv('ticker.csv', sep=";", index=False)

In [5]:
def get_stock_status(stock_name):
        base_URL = "https://statusinvest.com.br/acoes/"
        final_URL = base_URL+stock_name
        driver= webdriver.Chrome()
        print("acessando página de", stock_name)
        driver.get(final_URL)
        
        sleep(3.0)
        ativo_name = driver.find_element(By.XPATH, '//*[@id="main-header"]/div[2]/div/div[1]/h1/small')
        button_historico = driver.find_element(By.XPATH,'//*[@id="indicators-section"]/div[1]/div[2]/button[@title="Histórico do ativo"]')
        button_historico.click()
        print("clicando botão para acessar histórico")
        sleep(4.5)

        complete_tables = driver.find_elements(By.XPATH, '//div[@class="indicators"]')
        dic_list = []
        print("extraindo dados de", stock_name)
        for elem in complete_tables:
                indicator_names = elem.find_elements(By.TAG_NAME, 'h3')
                for i in range(len(indicator_names)):
                        year = elem.find_element(By.XPATH, '//*[@id="indicators-section"]/div[3]/div[2]/div/div[2]/div/div[1]').text.split('\n')
                        value = elem.find_element(By.XPATH, '//*[@id="indicators-section"]/div[3]/div[2]/div/div[2]/div/div[' +str(i+2) +']').text.split('\n')
                        for j in range(len(year)):
                                dic = {
                                        "ativo": stock_name,
                                        "indicator_name": indicator_names[i].text,
                                        "nome": ativo_name.text,
                                        "year":year[j],
                                        "value":value[j]
                                }
                                dic_list.append(dic)
        driver.close()
        return dumps(dic_list, indent = 6)

In [ ]:
ticker =  pd.read_csv('ticker.csv', sep=";")


for i in range(len(ticker)):
    if(ticker.at[i, 'Status']== 0):
        ativo = ticker.at[i, "TICKER"]
        print("proximo ativo", ativo)
        while True:
            try:
                sleep(1.0)
                data_to_save = get_stock_status(ativo)
            except:
                print("tentando", ativo, "novamente")
                continue
            break
        f = open("ativos/"+ativo, "w")
        f.write(data_to_save)
        f.close()
        ticker.at[i, 'Status'] = 1
        ticker.to_csv('ticker.csv', sep=";", index=False)



